In [2]:
import numpy as np
import pandas as pd
from scipy import spatial
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics.pairwise import pairwise_distances 
import sklearn

In [ ]:
# filename = 'u1.base'
# df = pd.read_csv(filename,sep='\t',names=['user_id','item_id','rating','timestamp'])
# del df['timestamp']
# df.head()

In [ ]:
# users = np.arange(df_user.shape[0]) + 1
# no_users, no_movies = df.user_id.unique().shape[0], df.item_id.unique().shape[0]
# no_users, no_movies = 943, 1682
# print(no_users, no_movies)

In [ ]:
# def isNaN(num):
#     return num != num

In [ ]:
# # users
# no_users, no_movies = 943, 1682
# entire_review = np.zeros((no_users, no_movies))
# for row in df.itertuples():
#     user, movie, rating = row[1], row[2], row[3]
#     entire_review[user - 1, movie - 1] = rating

In [ ]:
# entire_review.shape

In [ ]:
# Since similarity, we will use 1 - cosine_distance
# similarity_users = 1 - pairwise_distances(entire_review, metric='cosine')
# similarity_movies = 1 - pairwise_distances(entire_review.T, metric='cosine')

In [ ]:
# entire_review

In [ ]:
# df_train = pd.read_csv('u1.base',sep='\t',names=['user_id','item_id','rating','timestamp'])
# del df_train['timestamp']
# df_train.head()

In [ ]:
# df_test = pd.read_csv('u1.test',sep='\t',names=['user_id','item_id','rating','timestamp'])
# del df_test['timestamp']
# df_test.head()

In [ ]:
# y_true = np.array(df_test['rating'])
# y_pred = np.zeros(len(y_true))

In [ ]:
# def has_rated(user, movie, df):
#     for row in df.itertuples():
#         user_, movie_, rating = row[1], row[2], row[3]
#         if user == user and movie == movie_:
#             return True
#     return False

In [3]:
# User based system
filenames = [
    ['u1.base', 'u1.test'], 
    ['u2.base', 'u2.test'], 
    ['u3.base', 'u3.test'], 
    ['u4.base', 'u4.test'], 
    ['u5.base', 'u5.test']
]
thresholds = [0.4, 0.5, 0.6, 0.7]
# mae_val = 0

for threshold in thresholds:
    for filename in filenames:
        train_name, test_name = filename[0], filename[1]
        df_test = pd.read_csv(test_name,sep='\t',names=['user_id','item_id','rating','timestamp'])
        df_train = pd.read_csv(train_name,sep='\t',names=['user_id','item_id','rating','timestamp'])
        no_users, no_movies = 943, 1682
        entire_review = np.zeros((no_users, no_movies))
        has_rated = {}
        for row in df_train.itertuples():
            user, movie, rating = row[1], row[2], row[3]
            entire_review[user - 1, movie - 1] = rating
            if user not in has_rated:
                has_rated[user] = [movie]
            else:
                has_rated[user].append(movie)
#         Finding places where no review was taken
#         indices = np.where(entire_review == 2020)
#         listOfCoordinates = list(zip(indices[0], indices[1]))
#         for index in listOfCoordinates:
#             x, y = index[0], index[1]
#             entire_review[x, y] = 0
#         Subtracting row mean from each element for centered cosine-similarity
#         entire_review = entire_review - entire_review.mean(axis=1, keepdims=True)
#         for index in listOfCoordinates:
#             x, y = index[0], index[1]
# #             Now, 0 is actual average
#             entire_review[x, y] = 0
        y_true = np.array(df_test['rating'])
        y_pred = np.zeros(len(y_true))
        similarity_users = 1 - pairwise_distances(entire_review, metric='cosine')
#         similarity_movies = 1 - pairwise_distances(entire_review.T, metric='cosine')
        iterator = 0
        for row in df_test.itertuples():
            user, movie, rating = row[1], row[2], row[3]
        #     Get the user-user similarities for the current user
            temp = similarity_users[user - 1, :]
        #     Remove the current user from users to check
            temp = np.delete(temp, user - 1)
        #     print(temp)
        #     Assumption that users should be greater than or equal to threshold
            users_to_check = np.ndarray.flatten(np.array(np.where(temp > threshold))) + 1
            final_users = []
            for us in users_to_check:
                if movie in has_rated[us]:
                    final_users.append(us)
            users_to_check = np.array(final_users)
            if users_to_check.shape[0] == 0:
                y_pred[iterator] = 3
                iterator += 1
                continue
        #     print(users_to_check)
            ratings = np.zeros(users_to_check.shape[0])
        #     The normalising denominator
            denom = np.sum(np.array([similarity_users[user - 1, i - 1] for i in users_to_check]))
            index = 0
            for i in range(users_to_check.shape[0]):
        #         a, b = entire_review[i - 1, movie - 1], similarity_users[user - 1, i - 1]
        #         ans += a*b/denom
                rating_by_user = entire_review[users_to_check[i] - 1, movie - 1]
        #         print('Rating', rating_by_user)
                sim = similarity_users[user - 1, users_to_check[i] - 1]
        #         print('Similarity', sim)
        #         print(users_to_check[i] - 1)
        #         print(movie - 1)
        #         print(entire_review[users_to_check[i] - 1, movie - 1])
                if sim == 0:
                    ratings[index] = 0
                else:
                    ratings[index] = rating_by_user * sim / denom 
#                 if isNaN(ratings[index]):
#                     print(rating_by_user, sim)
#                     ratings[index]
#                     break
                index += 1
        #     ratings /= denom
        #     mae_val = abs(rating - ans)
            y_pred[iterator] = int(round(np.sum(ratings)))
            iterator += 1
        #     print(np.sum(ratings))
        #     print(users_to_check)
        #     print(denom)
        #     break
        print('Threshold', threshold)
        print('Train/Test file checked', filename)
        print('MAE Value', mae(y_true, y_pred))

Threshold 0.4
Train/Test file checked ['u1.base', 'u1.test']
MAE Value 1.0297
Threshold 0.4
Train/Test file checked ['u2.base', 'u2.test']
MAE Value 0.96135
Threshold 0.4
Train/Test file checked ['u3.base', 'u3.test']
MAE Value 0.9086
Threshold 0.4
Train/Test file checked ['u4.base', 'u4.test']
MAE Value 0.8943
Threshold 0.4
Train/Test file checked ['u5.base', 'u5.test']
MAE Value 0.9223
Threshold 0.5
Train/Test file checked ['u1.base', 'u1.test']
MAE Value 1.0332
Threshold 0.5
Train/Test file checked ['u2.base', 'u2.test']
MAE Value 1.01445
Threshold 0.5
Train/Test file checked ['u3.base', 'u3.test']
MAE Value 0.97425
Threshold 0.5
Train/Test file checked ['u4.base', 'u4.test']
MAE Value 0.9684
Threshold 0.5
Train/Test file checked ['u5.base', 'u5.test']
MAE Value 0.9798
Threshold 0.6
Train/Test file checked ['u1.base', 'u1.test']
MAE Value 1.0333
Threshold 0.6
Train/Test file checked ['u2.base', 'u2.test']
MAE Value 1.0138
Threshold 0.6
Train/Test file checked ['u3.base', 'u3.test']


In [11]:
# Item based system
filenames = [
    ['u1.base', 'u1.test'], 
    ['u2.base', 'u2.test'], 
    ['u3.base', 'u3.test'], 
    ['u4.base', 'u4.test'], 
    ['u5.base', 'u5.test']
]
k_vals = [10, 20, 30, 40]
# mae_val = 0

for k in k_vals:
    for filename in filenames:
        train_name, test_name = filename[0], filename[1]
        df_test = pd.read_csv(test_name,sep='\t',names=['user_id','item_id','rating','timestamp'])
        df_train = pd.read_csv(train_name,sep='\t',names=['user_id','item_id','rating','timestamp'])
        no_users, no_movies = 943, 1682
        entire_review = np.zeros((no_users, no_movies)) + 3
        for row in df_train.itertuples():
            user, movie, rating = row[1], row[2], row[3]
            entire_review[user - 1, movie - 1] = rating
# #         Finding places where no review was taken
#         indices = np.where(entire_review == 2020)
#         listOfCoordinates = list(zip(indices[0], indices[1]))
#         for index in listOfCoordinates:
#             x, y = index[0], index[1]
#             entire_review[x, y] = 0
# #         Subtracting row mean from each element for centered cosine-similarity
#         entire_review = entire_review - entire_review.mean(axis=1, keepdims=True)
#         for index in listOfCoordinates:
#             x, y = index[0], index[1]
# #             Now, 0 is actual average
#             entire_review[x, y] = 0
#         for row in df_train.itertuples():
#             user, movie, rating = row[1], row[2], row[3]
#             entire_review[user - 1, movie - 1] = rating
#         y_true = np.array(df_test['rating'])
        y_pred = np.zeros(len(y_true))
#         similarity_users = 1 - pairwise_distances(entire_review, metric='cosine')
        similarity_movies = 1 - pairwise_distances(entire_review.T, metric='cosine')
        iterator = 0
        for row in df_test.itertuples():
            user, movie, rating = row[1], row[2], row[3]
        #     Get the item-item similarities for the current user
            temp = similarity_movies[movie - 1, :]
#             denom = np.sum(temp)
        #     Remove the current movie from users to check
            temp = np.delete(temp, movie - 1)
        #     print(temp)
        #     Assumption that users should be greater than or equal to threshold
            temp = np.argpartition(-temp, k)
            temp = temp[:4]
#             idx = (-temp).argsort()[:k]
            movies_to_check = np.array(temp) + 1
#             print(movies_to_check)
#             if users_to_check.shape[0] == 0:
#                 y_pred[iterator] = 3
#                 iterator += 1
#                 continue
        #     print(users_to_check)
            ratings = np.zeros(movies_to_check.shape[0])
        #     The normalising denominator
            denom = np.sum(np.array([similarity_movies[movie - 1, i - 1] for i in movies_to_check]))
            index = 0
            for i in range(movies_to_check.shape[0]):
        #         a, b = entire_review[i - 1, movie - 1], similarity_users[user - 1, i - 1]
        #         ans += a*b/denom
                rating_by_user = entire_review[user - 1, movies_to_check[i] - 1]
        #         print('Rating', rating_by_user)
                sim = similarity_movies[movie - 1, movies_to_check[i] - 1]
        #         print('Similarity', sim)
        #         print(users_to_check[i] - 1)
        #         print(movie - 1)
        #         print(entire_review[users_to_check[i] - 1, movie - 1])
                if sim == 0:
                    ratings[index] = 0
                else:
                    ratings[index] = rating_by_user * sim / denom 
#                 if isNaN(ratings[index]):
#                     print(rating_by_user, sim)
#                     ratings[index]
#                     break
                index += 1
        #     ratings /= denom
        #     mae_val = abs(rating - ans)
            y_pred[iterator] = int(round(np.sum(ratings)))
            iterator += 1
        #     print(np.sum(ratings))
        #     print(users_to_check)
        #     print(denom)
        #     break
        print('K value', k)
        print('Train/Test file checked', filename)
        print('MAE Value', mae(y_true, y_pred))

K value 10
Train/Test file checked ['u1.base', 'u1.test']
MAE Value 0.99565
K value 10
Train/Test file checked ['u2.base', 'u2.test']
MAE Value 0.9942
K value 10
Train/Test file checked ['u3.base', 'u3.test']
MAE Value 0.9959
K value 10
Train/Test file checked ['u4.base', 'u4.test']
MAE Value 0.994
K value 10
Train/Test file checked ['u5.base', 'u5.test']
MAE Value 0.9872
K value 20
Train/Test file checked ['u1.base', 'u1.test']
MAE Value 0.9954
K value 20
Train/Test file checked ['u2.base', 'u2.test']
MAE Value 0.99465
K value 20
Train/Test file checked ['u3.base', 'u3.test']
MAE Value 0.9944
K value 20
Train/Test file checked ['u4.base', 'u4.test']
MAE Value 0.9944
K value 20
Train/Test file checked ['u5.base', 'u5.test']
MAE Value 0.9884
K value 30
Train/Test file checked ['u1.base', 'u1.test']
MAE Value 0.9956
K value 30
Train/Test file checked ['u2.base', 'u2.test']
MAE Value 0.99505
K value 30
Train/Test file checked ['u3.base', 'u3.test']
MAE Value 0.99425
K value 30
Train/Test 

In [ ]:
# similarity = [[0]*no_users for i in range(no_users)]
# similarity = np.array(similarity)

In [ ]:
def error(a, b):
    err = 0
    for i in range(len(a)):
        err += (abs(a[i] - b[i]) / len(a))
#         print(err)
#     err /= len(a)
    return err

In [ ]:
df_test = pd.read_csv('u1.test',sep='\t',names=['user_id','item_id','rating','timestamp'])
# del df_test['timestamp']
y_true = np.array(df_test['rating'])
y_pred = np.zeros(len(y_true))
iterator = 0
threshold = 0.5
for row in df_test.itertuples():
    user, movie, rating = row[1], row[2], row[3]
#     Get the user-user similarities for the current user
    temp = similarity_users[user - 1, :]
#     Remove the current user from users to check
    temp = np.delete(temp, user - 1)
#     print(temp)
#     Assumption that users should be greater than or equal to threshold
    users_to_check = np.ndarray.flatten(np.array(np.where(temp > threshold))) + 1
#     print(users_to_check)
#     If no user satisfies criteria
    if users_to_check.shape[0] == 0:
        y_pred[iterator] = 0
        iterator += 1
        continue
    ratings = np.zeros(users_to_check.shape[0])
#     The normalising denominator
    denom = np.sum(np.array([similarity_users[user - 1, i - 1] for i in users_to_check]))
    index = 0
    for i in range(users_to_check.shape[0]):
#         a, b = entire_review[i - 1, movie - 1], similarity_users[user - 1, i - 1]
#         ans += a*b/denom
        rating_by_user = entire_review[users_to_check[i] - 1, movie - 1]
#         print('Rating', rating_by_user)
        sim = similarity_users[user - 1, users_to_check[i] - 1]
#         print('Similarity', sim)
#         print(users_to_check[i] - 1)
#         print(movie - 1)
#         print(entire_review[users_to_check[i] - 1, movie - 1])
        ratings[index] = rating_by_user * sim / denom 
        index += 1
#     ratings /= denom
#     mae_val = abs(rating - ans)
#     print(ratings)
    y_pred[iterator] = int(round(np.sum(ratings)))
    iterator += 1
#     print(np.sum(ratings))
#     print(users_to_check)
#     print(denom)
#     break
print('Threshold', threshold)
print('Test file checked', filename)
# print(type(y_pred))
print('MAE Value', error(y_pred, y_true))
print(y_true.shape)
print(y_pred.shape)
count = 0
for i in range(len(y_pred)):
    if y_pred[i] == 0:
        count += 1
print(count)

In [ ]:
# df.shape

In [ ]:
# df_test.shape

In [ ]:
# entire_review = np.zeros()

In [ ]:
# df.loc[(df['user_id'] == 1) & (df['item_id'] == 10)]

In [ ]:
# def sim(a, b):
#     return 1 - spatial.distance.cosine(a, b)

In [ ]:
# for 

In [ ]:
# # User based
# thresholds = [0.4, 0.5, 0.6, 0.7]
# threshold = 0.7
# # for threshold in thresholds:
# y_true = df_test['rating']
# for index, row in df_test.iterrows():
#     user = 